In [1]:
#모듈
import math
import pandas as pd
import numpy as np

try:
    import cufflinks as cf
except:
    !pip install cufflinks
    import cufflinks as cf

cf.go_offline()

from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [2]:
# 데이터 다루기
pd.set_option("Display.max_columns",200)
import re
import time

# 데이터 시각화
import seaborn as sns

# 사용할 머신러닝 모델들 - RandomForest, AdaBoost, GradientBoost, ExtraTrees
import sklearn
from sklearn.metrics import log_loss
from sklearn.svm import SVC

In [3]:
#명목형 변수 더미화
def dummify_category_cols(df,cols):
    dummies = []
    new_df = df.copy()

    for idx in range(len(cols)):
        new_df = new_df.join(pd.get_dummies(total[cols[idx]], prefix = cols[idx]))
        new_df = new_df.drop(cols[idx], axis =1)
    return new_df

In [4]:
#숫자형 변수 범주화
def dummify_number_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: int(x)//num * num)
    return new_df


In [5]:
#명목형 변수의 연속형 변환 - 변수의 차원을 낮추기 위하여
def num_category(df, col, chg) :
    new_df = df.copy()
    set = {}
    
    for idx in range(len(chg)) :
        set[chg[idx]] = idx
        
    new_df[col] = new_df[col].map(set)
    
    return new_df        

In [6]:
#숫자형 변수 일정 이상은 묶어버리기
def number_group_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: num if x> num else x )
    return new_df


In [7]:
#RMS 구하기
def RMSE(y_true,y_pred):
    
    squaredErrors = (y_pred - y_true)**2
    MSE = squaredErrors.mean()
    
    return math.sqrt(MSE)

In [8]:
#본격적으로 시작

In [9]:
training = pd.read_csv("house_price.csv")
test = pd.read_csv("test.csv")

#training = training[np.isfinite(training["GarageYrBlt"])]

In [10]:
total = pd.concat([training, test]).reset_index()
del total["index"]

In [11]:
total.head(2)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,Condition1,Condition2,Electrical,EnclosedPorch,ExterCond,ExterQual,Exterior1st,Exterior2nd,Fence,FireplaceQu,Fireplaces,Foundation,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,GrLivArea,HalfBath,Heating,HeatingQC,HouseStyle,Id,KitchenAbvGr,KitchenQual,LandContour,LandSlope,LotArea,LotConfig,LotFrontage,LotShape,LowQualFinSF,MSSubClass,MSZoning,MasVnrArea,MasVnrType,MiscFeature,MiscVal,MoSold,Neighborhood,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,PoolQC,RoofMatl,RoofStyle,SaleCondition,SalePrice,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,NaN,3,1Fam,TA,No,706.0,0.0,GLQ,Unf,1.0,0.0,Gd,150.0,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,NaN,NaN,0,PConc,2,Typ,548.0,2.0,TA,RFn,TA,Attchd,2003.0,1710,1,GasA,Ex,2Story,1,1,Gd,Lvl,Gtl,8450,Inside,65.0,Reg,0,60,RL,196.0,BrkFace,NaN,0,2,CollgCr,61,5,7,Y,0,NaN,CompShg,Gable,Normal,208500.0,WD,0,Pave,8,856.0,AllPub,0,2003,2003,2008
1,1262,0,0,NaN,3,1Fam,TA,Gd,978.0,0.0,ALQ,Unf,0.0,1.0,Gd,284.0,Y,Feedr,Norm,SBrkr,0,TA,TA,MetalSd,MetalSd,NaN,TA,1,CBlock,2,Typ,460.0,2.0,TA,RFn,TA,Attchd,1976.0,1262,0,GasA,Ex,1Story,2,1,TA,Lvl,Gtl,9600,FR2,80.0,Reg,0,20,RL,0.0,None,NaN,0,5,Veenker,0,8,6,Y,0,NaN,CompShg,Gable,Normal,181500.0,WD,0,Pave,6,1262.0,AllPub,298,1976,1976,2007


In [12]:
total.columns

Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'Alley', 'BedroomAbvGr',
       'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtFinType1', 'BsmtFinType2', 'BsmtFullBath', 'BsmtHalfBath',
       'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'EnclosedPorch', 'ExterCond', 'ExterQual', 'Exterior1st',
       'Exterior2nd', 'Fence', 'FireplaceQu', 'Fireplaces', 'Foundation',
       'FullBath', 'Functional', 'GarageArea', 'GarageCars', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'GarageYrBlt', 'GrLivArea',
       'HalfBath', 'Heating', 'HeatingQC', 'HouseStyle', 'Id', 'KitchenAbvGr',
       'KitchenQual', 'LandContour', 'LandSlope', 'LotArea', 'LotConfig',
       'LotFrontage', 'LotShape', 'LowQualFinSF', 'MSSubClass', 'MSZoning',
       'MasVnrArea', 'MasVnrType', 'MiscFeature', 'MiscVal', 'MoSold',
       'Neighborhood', 'OpenPorchSF', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolArea', 'Po

In [67]:
#선택된 변수
category_selected = ["MSSubClass", "MSZoning", "Street" ,"LandContour", "LotConfig", 
            "Neighborhood", "BldgType", "HouseStyle", "OverallQual", 
            "MasVnrType", "ExterQual", "ExterCond",
            "Foundation","BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "HeatingQC", "CentralAir",
            "Electrical", "BsmtFullBath", "FullBath", "HalfBath", "KitchenQual",
            "TotRmsAbvGrd", "Functional", "Fireplaces", "FireplaceQu", "GarageType",
            "GarageCars", "PavedDrive", "SaleCondition", "GarageFinish"]

extra_selected = [ "GarageCars", "Electrical" ,"OverallQual","OverallCond","ExterQual", "ExterCond",  "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "HeatingQC","KitchenQual","FireplaceQu", "Functional"]

number_selected = ["GarageYrBlt","YearBuilt", "YearRemodAdd", "BsmtFinSF1", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "GrLivArea", 'GarageArea', "LotArea", "MasVnrArea", "WoodDeckSF", "OpenPorchSF"]

category_selected = [x for x in category_selected if x not in extra_selected]

total_selected = category_selected + number_selected + extra_selected

selected_with_SalePrice = total_selected +["SalePrice"]

In [68]:
new_total = total[selected_with_SalePrice].copy()
new_total.head(2)

,MSSubClass,MSZoning,Street,LandContour,LotConfig,Neighborhood,BldgType,HouseStyle,MasVnrType,Foundation,CentralAir,BsmtFullBath,FullBath,HalfBath,TotRmsAbvGrd,Fireplaces,GarageType,PavedDrive,SaleCondition,GarageFinish,GarageYrBlt,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,LotArea,MasVnrArea,WoodDeckSF,OpenPorchSF,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice
0,60,RL,Pave,Lvl,Inside,CollgCr,1Fam,2Story,BrkFace,PConc,Y,1.0,2,1,8,0,Attchd,Y,Normal,RFn,2003.0,2003,2003,706.0,150.0,856.0,856,854,1710,548.0,8450,196.0,0,61,2.0,SBrkr,7,5,Gd,TA,Gd,TA,No,GLQ,Ex,Gd,NaN,Typ,208500.0
1,20,RL,Pave,Lvl,FR2,Veenker,1Fam,1Story,None,CBlock,Y,0.0,2,0,6,1,Attchd,Y,Normal,RFn,1976.0,1976,1976,978.0,284.0,1262.0,1262,0,1262,460.0,9600,0.0,298,0,2.0,SBrkr,6,8,TA,TA,Gd,TA,Gd,ALQ,Ex,TA,TA,Typ,181500.0


In [69]:
#임시 전처리

In [70]:
#전처리1 - 연속형데이터 중 null값이 있는 행 모두 버려주기
new_total[number_selected].isnull().sum()

GarageYrBlt     159
YearBuilt         0
YearRemodAdd      0
BsmtFinSF1        1
BsmtUnfSF         1
TotalBsmtSF       1
1stFlrSF          0
2ndFlrSF          0
GrLivArea         0
GarageArea        1
LotArea           0
MasVnrArea       23
WoodDeckSF        0
OpenPorchSF       0
dtype: int64

In [71]:
new_total["YearBuilt"] = new_total["YearBuilt"].fillna(new_total.YearBuilt.mean())

In [72]:
new_total["BsmtFinSF1"] = new_total["BsmtFinSF1"].fillna(new_total.BsmtFinSF1.mean())

In [73]:
new_total["BsmtUnfSF"] = new_total["BsmtUnfSF"].fillna(new_total.BsmtUnfSF.mean())

In [74]:
new_total["TotalBsmtSF"] = new_total["TotalBsmtSF"].fillna(new_total.TotalBsmtSF.mean())

In [75]:
new_total["GarageArea"] = new_total["GarageArea"].fillna(new_total.GarageArea.mean())

In [76]:
new_total["GarageYrBlt"] = new_total["GarageYrBlt"].fillna(new_total.GarageYrBlt.mean())

In [77]:
new_total["Electrical"] = new_total["Electrical"].fillna("SBrkr")

In [78]:
new_total = new_total.fillna("404")

In [79]:
new_total.isnull().sum()

MSSubClass       0
MSZoning         0
Street           0
LandContour      0
LotConfig        0
Neighborhood     0
BldgType         0
HouseStyle       0
MasVnrType       0
Foundation       0
CentralAir       0
BsmtFullBath     0
FullBath         0
HalfBath         0
TotRmsAbvGrd     0
Fireplaces       0
GarageType       0
PavedDrive       0
SaleCondition    0
GarageFinish     0
GarageYrBlt      0
YearBuilt        0
YearRemodAdd     0
BsmtFinSF1       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
GrLivArea        0
GarageArea       0
LotArea          0
MasVnrArea       0
WoodDeckSF       0
OpenPorchSF      0
GarageCars       0
Electrical       0
OverallQual      0
OverallCond      0
ExterQual        0
ExterCond        0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
HeatingQC        0
KitchenQual      0
FireplaceQu      0
Functional       0
SalePrice        0
dtype: int64

In [80]:
new_total.head()

,MSSubClass,MSZoning,Street,LandContour,LotConfig,Neighborhood,BldgType,HouseStyle,MasVnrType,Foundation,CentralAir,BsmtFullBath,FullBath,HalfBath,TotRmsAbvGrd,Fireplaces,GarageType,PavedDrive,SaleCondition,GarageFinish,GarageYrBlt,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,LotArea,MasVnrArea,WoodDeckSF,OpenPorchSF,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice
0,60,RL,Pave,Lvl,Inside,CollgCr,1Fam,2Story,BrkFace,PConc,Y,1,2,1,8,0,Attchd,Y,Normal,RFn,2003.0,2003,2003,706.0,150.0,856.0,856,854,1710,548.0,8450,196,0,61,2,SBrkr,7,5,Gd,TA,Gd,TA,No,GLQ,Ex,Gd,404,Typ,208500
1,20,RL,Pave,Lvl,FR2,Veenker,1Fam,1Story,None,CBlock,Y,0,2,0,6,1,Attchd,Y,Normal,RFn,1976.0,1976,1976,978.0,284.0,1262.0,1262,0,1262,460.0,9600,0,298,0,2,SBrkr,6,8,TA,TA,Gd,TA,Gd,ALQ,Ex,TA,TA,Typ,181500
2,60,RL,Pave,Lvl,Inside,CollgCr,1Fam,2Story,BrkFace,PConc,Y,1,2,1,6,1,Attchd,Y,Normal,RFn,2001.0,2001,2002,486.0,434.0,920.0,920,866,1786,608.0,11250,162,0,42,2,SBrkr,7,5,Gd,TA,Gd,TA,Mn,GLQ,Ex,Gd,TA,Typ,223500
3,70,RL,Pave,Lvl,Corner,Crawfor,1Fam,2Story,None,BrkTil,Y,1,1,0,7,1,Detchd,Y,Abnorml,Unf,1998.0,1915,1970,216.0,540.0,756.0,961,756,1717,642.0,9550,0,0,35,3,SBrkr,7,5,TA,TA,TA,Gd,No,ALQ,Gd,Gd,Gd,Typ,140000
4,60,RL,Pave,Lvl,FR2,NoRidge,1Fam,2Story,BrkFace,PConc,Y,1,2,1,9,1,Attchd,Y,Normal,RFn,2000.0,2000,2000,655.0,490.0,1145.0,1145,1053,2198,836.0,14260,350,192,84,3,SBrkr,8,5,Gd,TA,Gd,TA,Av,GLQ,Ex,Gd,TA,Typ,250000


In [81]:
#명목형 변수를 연속형으로 바꿔주는 것

In [82]:
new_total = num_category(new_total, "ExterQual", ["Po", "Fa", "TA", "Gd", "Ex"])

In [83]:
new_total = num_category(new_total, "ExterCond", ["Po", "Fa", "TA", "Gd", "Ex"])

In [84]:
new_total = num_category(new_total, "BsmtQual", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [85]:
new_total = num_category(new_total, "BsmtCond", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [86]:
new_total = num_category(new_total, "BsmtExposure", ["404", "No", "Fa", "Mn", "Av", "Gd"])

In [87]:
new_total = num_category(new_total, "BsmtFinType1", ["404", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"])

In [88]:
new_total = num_category(new_total, "HeatingQC", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [89]:
new_total = num_category(new_total, "KitchenQual", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [90]:
new_total = num_category(new_total, "FireplaceQu", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [91]:
new_total = num_category(new_total, "Electrical", ["Mix", "FuseP", "FuseF", "FuseA", "SBrkr"])

In [92]:
new_total = num_category(new_total, "Functional", ["404", "Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"])
new_total.Functional.unique()

array([8, 7, 4, 6, 5, 3, 2, 0], dtype=int64)

In [93]:
#변수끼리 합쳐주는 것
new_total["Functional"] = total["Functional"].map(lambda x : 1 if x==8 else 0)
new_total["LotConfig"] = new_total.LotConfig.map(lambda x : 1 if x == "CulDSac" else 0)
new_total["FullBath"] = new_total.FullBath.map(lambda x : 1 if x <=2 else x-1)
new_total["OverallCond"] = new_total.OverallCond.map(lambda x : 1 if x <=4 else 2)
new_total["BldgType"] = new_total.BldgType.map(lambda x : 1 if x =="1Fam" else 2)
new_total = number_group_cols(new_total, ["TotRmsAbvGrd"], [13])
new_total = number_group_cols(new_total, ["HalfBath"], [1])
new_total = number_group_cols(new_total, ["FullBath"], [2])

In [94]:
new_total["Functional"] = total["Functional"].map(lambda x : 1 if x==8 else 0)
new_total["LotConfig"] = new_total.LotConfig.map(lambda x : 1 if x == "CulDSac" else 0)
new_total["FullBath"] = new_total.FullBath.map(lambda x : 1 if x <=2 else x-1)
new_total["OverallCond"] = new_total.OverallCond.map(lambda x : 1 if x <=4 else 2)
new_total["BldgType"] = new_total.BldgType.map(lambda x : 1 if x =="1Fam" else 2)
new_total = number_group_cols(new_total, ["TotRmsAbvGrd"], [13])
new_total = number_group_cols(new_total, ["HalfBath"], [1])
new_total = number_group_cols(new_total, ["FullBath"], [2])

In [95]:
#임시 전처리

In [96]:
new_total = new_total.assign(Is_Remod =  lambda x : x["YearBuilt"] != x["YearRemodAdd"])
new_total["Is_Remod"] = new_total.YearRemodAdd.map(lambda x : 1 if x == True else 0)
new_total["after"] = 2011 - new_total["YearRemodAdd"]
new_total = new_total.drop("YearRemodAdd", axis = 1)

In [97]:
new_total = new_total.assign(Is_bsmt_unfinish = lambda x : x["BsmtFinType1"] == 0)
new_total["Is_bsmt_unfinish"] = new_total.Is_bsmt_unfinish.map(lambda x : 1 if x == True else 0)

In [98]:
new_total = new_total.assign(Is_Fireplaces = lambda x : x["Fireplaces"] != 0)
new_total["Is_Fireplaces"] = new_total.Is_Fireplaces.map(lambda x : 1 if x == True else 0)

In [99]:
new_total = new_total.assign(Is_GarageFinish = lambda x : x["GarageFinish"] != "Unf")
new_total["Is_GarageFinish "] = new_total.Is_GarageFinish .map(lambda x : 1 if x == True else 0)
new_total = new_total.drop(["BsmtFinType1", "Fireplaces", "GarageFinish"], axis = 1)

In [100]:
new_total["Total_Area"] = new_total["GrLivArea"] + new_total["LotArea"] + new_total["GarageArea"] + new_total["TotalBsmtSF"]

In [101]:
new_total["GrLivArea"] = new_total["GrLivArea"].map(lambda x : math.log(x))

In [102]:
new_total["LotArea"] = new_total["LotArea"].map(lambda x : math.log(x))

In [103]:
new_total["GarageArea"] = new_total["GarageArea"].map(lambda x : math.log(x+1))

In [104]:
new_total["TotalBsmtSF"] = new_total["TotalBsmtSF"].map(lambda x : math.log(x+1))

In [105]:
new_total["Total_Area"] = new_total["Total_Area"].map(lambda x : math.log(x+1))

In [106]:
new_total["BsmtFinSF1"] = new_total["BsmtFinSF1"].map(lambda x : math.log(x+1))

In [107]:
new_total["BsmtUnfSF"] = new_total["BsmtUnfSF"].map(lambda x : math.log(x+1))

In [108]:
new_total["1stFlrSF"] = new_total["1stFlrSF"].map(lambda x : math.log(x+1))

In [109]:
new_total["2ndFlrSF"] = new_total["2ndFlrSF"].map(lambda x : math.log(x+1))

In [110]:
new_total["is_masvnr"] = new_total.MasVnrArea.map(lambda x : 0 if int(x) == 0 else 1)
new_total["is_wood"] = new_total.WoodDeckSF.map(lambda x : 0 if int(x) == 0 else 1)
new_total["is_openporch"] = new_total.OpenPorchSF.map(lambda x : 0 if int(x) == 0 else 1)

In [111]:
new_total["MasVnrArea"] = new_total["MasVnrArea"].map(lambda x : math.log(int(x)+1))
new_total["WoodDeckSF"] = new_total["WoodDeckSF"].map(lambda x : math.log(int(x)+1))
new_total["OpenPorchSF"] = new_total["OpenPorchSF"].map(lambda x : math.log(int(x)+1))

In [112]:
new_total["SalePrice"] = new_total.SalePrice.map(lambda x: math.log(int(x)+1))

In [113]:
category_selected = [x for x in category_selected if x not in["BsmtFinType1", "Fireplaces", "GarageFinish"]]

In [114]:
#확인

In [115]:
new_total.head(2)

,MSSubClass,MSZoning,Street,LandContour,LotConfig,Neighborhood,BldgType,HouseStyle,MasVnrType,Foundation,CentralAir,BsmtFullBath,FullBath,HalfBath,TotRmsAbvGrd,GarageType,PavedDrive,SaleCondition,GarageYrBlt,YearBuilt,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,LotArea,MasVnrArea,WoodDeckSF,OpenPorchSF,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice,Is_Remod,after,Is_bsmt_unfinish,Is_Fireplaces,Is_GarageFinish,Is_GarageFinish,Total_Area,is_masvnr,is_wood,is_openporch
0,60,RL,Pave,Lvl,0,CollgCr,2,2Story,BrkFace,PConc,Y,1,1,1,8,Attchd,Y,Normal,2003.0,2003,6.561031,5.017280,6.753438,6.753438,6.751101,7.444249,6.308098,9.041922,5.283204,0.000000,4.127134,2,4,7,1,3,2,4,3,1,5,4,0,0,12.247699,0,8,0,0,True,1,9.355739,1,0,1
1,20,RL,Pave,Lvl,0,Veenker,2,1Story,None,CBlock,Y,0,1,0,6,Attchd,Y,Normal,1976.0,1976,6.886532,5.652489,7.141245,7.141245,0.000000,7.140453,6.133398,9.169518,0.000000,5.700444,0.000000,2,4,6,1,2,2,4,3,5,5,3,3,0,12.109016,0,35,0,1,True,1,9.440261,0,1,0


In [116]:
#히트맵으로 확인
#training = training = new_total.iloc[:len(training)]

In [117]:
#n= len(selected_with_SalePrice)
#colormap = plt.cm.viridis
#plt.figure(figsize=(n,n))
#plt.title('Pearson Correlation of Features', y=1.05, size=15)
#sns.heatmap(training.corr(),
#            linewidths=0.1,
#            vmax=1.0, 
#            square=True, 
#            cmap=colormap, 
#            linecolor='white', 
#            annot=True)

In [123]:
new_total2 = dummify_category_cols(new_total,category_selected)

In [124]:
new_total2.head(2)

,GarageYrBlt,YearBuilt,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,LotArea,MasVnrArea,WoodDeckSF,OpenPorchSF,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice,Is_Remod,after,Is_bsmt_unfinish,Is_Fireplaces,Is_GarageFinish,Is_GarageFinish,Total_Area,is_masvnr,is_wood,is_openporch,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,CentralAir_N,CentralAir_Y,BsmtFullBath_0.0,BsmtFullBath_1.0,BsmtFullBath_2.0,BsmtFullBath_3.0,FullBath_0,FullBath_1,FullBath_2,FullBath_3,FullBath_4,HalfBath_0,HalfBath_1,HalfBath_2,TotRmsAbvGrd_2,TotRmsAbvGrd_3,TotRmsAbvGrd_4,TotRmsAbvGrd_5,TotRmsAbvGrd_6,TotRmsAbvGrd_7,TotRmsAbvGrd_8,TotRmsAbvGrd_9,TotRmsAbvGrd_10,TotRmsAbvGrd_11,TotRmsAbvGrd_12,TotRmsAbvGrd_13,TotRmsAbvGrd_14,TotRmsAbvGrd_15,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,2003.0,2003,6.561031,5.017280,6.753438,6.753438,6.751101,7.444249,6.308098,9.041922,5.283204,0.000000,4.127134,2,4,7,1,3,2,4,3,1,5,4,0,0,12.247699,0,8,0,0,True,1,9.355739,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
1,1976.0,1976,6.886532,5.652489,7.141245,7.141245,0.000000,7.140453,6.133398,9.169518,0.000000,5.700444,0.000000,2,4,6,1,2,2,4,3,5,5,3,3,0,12.109016,0,35,0,1,True,1,9.440261,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0


In [125]:
training = new_total2.iloc[:len(training)]
test = new_total2.iloc[len(training):]

In [126]:
#여기서 selected 재설정

In [127]:
training.head(2)

,GarageYrBlt,YearBuilt,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,LotArea,MasVnrArea,WoodDeckSF,OpenPorchSF,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice,Is_Remod,after,Is_bsmt_unfinish,Is_Fireplaces,Is_GarageFinish,Is_GarageFinish,Total_Area,is_masvnr,is_wood,is_openporch,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,CentralAir_N,CentralAir_Y,BsmtFullBath_0.0,BsmtFullBath_1.0,BsmtFullBath_2.0,BsmtFullBath_3.0,FullBath_0,FullBath_1,FullBath_2,FullBath_3,FullBath_4,HalfBath_0,HalfBath_1,HalfBath_2,TotRmsAbvGrd_2,TotRmsAbvGrd_3,TotRmsAbvGrd_4,TotRmsAbvGrd_5,TotRmsAbvGrd_6,TotRmsAbvGrd_7,TotRmsAbvGrd_8,TotRmsAbvGrd_9,TotRmsAbvGrd_10,TotRmsAbvGrd_11,TotRmsAbvGrd_12,TotRmsAbvGrd_13,TotRmsAbvGrd_14,TotRmsAbvGrd_15,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,2003.0,2003,6.561031,5.017280,6.753438,6.753438,6.751101,7.444249,6.308098,9.041922,5.283204,0.000000,4.127134,2,4,7,1,3,2,4,3,1,5,4,0,0,12.247699,0,8,0,0,True,1,9.355739,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
1,1976.0,1976,6.886532,5.652489,7.141245,7.141245,0.000000,7.140453,6.133398,9.169518,0.000000,5.700444,0.000000,2,4,6,1,2,2,4,3,5,5,3,3,0,12.109016,0,35,0,1,True,1,9.440261,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0


In [128]:
training['is_train'] = np.random.uniform(0, 1, len(training)) <= .75

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [129]:
training_set = training[training['is_train']==True]
test_set = training[training['is_train']==False]

In [130]:
N = 5000
forest = RandomForestRegressor(n_estimators=N, n_jobs=2)

In [131]:
feature = new_total2.columns.drop("SalePrice")
target = ["SalePrice"]

In [132]:
forest.fit(training_set[feature], training_set[target])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=5000, n_jobs=2, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [133]:
forest.predict(test_set[feature])

array([ 12.61370776,  11.7140232 ,  11.91327566,  11.90566236,
        12.69369101,  12.34227075,  11.84311102,  12.46155067,
        11.75175536,  12.25240367,  11.75848123,  11.72875312,
        12.00200572,  12.22925439,  12.23680564,  12.77976391,
        11.98672578,  11.72285776,  12.15791016,  11.88087638,
        11.46419281,  11.73155452,  11.56628789,  11.69967323,
        11.47580001,  11.66427245,  12.06109633,  12.33855872,
        12.15905352,  12.07865443,  12.43126362,  11.5631976 ,
        12.22000205,  12.01402695,  11.84059223,  11.77795299,
        11.84781466,  12.36231536,  12.27135857,  11.92949311,
        11.51092364,  11.97862144,  11.5425698 ,  11.51795499,
        11.94254541,  12.10385892,  12.20722902,  12.99642897,
        11.68080529,  11.88936968,  12.46813712,  12.08797429,
        12.20549497,  11.95658422,  11.72220651,  11.88545729,
        11.46818483,  12.04760616,  12.31189457,  12.30126707,
        12.16275951,  11.6637179 ,  12.98104892,  11.58

In [134]:
forest.score(test_set[feature], test_set[target])

0.89010420538614421

In [135]:
y_pred=forest.predict(test_set[feature])

In [136]:
y_pred = np.array(list(map(lambda x: math.exp(x)-1,y_pred)))

In [137]:
y_true = test_set["SalePrice"]
y_ture= np.array(list(map(lambda x: math.exp(x)-1,y_true)))

In [138]:
y_true = test_set["SalePrice"]
rmse = RMSE(y_true,y_pred)
print(rmse)

185312.23696705967


In [139]:
importance = pd.DataFrame(forest.feature_importances_, index = training_set[feature].columns)

In [140]:
(
    importance.assign(header = lambda x: x.index.str.split("_").str[0])
                  .groupby("header")[0]
                  .sum()
                  .iplot(kind="bar")
)

In [72]:
#최적화

In [ ]:
# 최적화할 파라미터 N
best_score = 0
for N in range(4000,8500,500):
    tmp = RandomForestRegressor(n_estimators=N)
    tmp.fit(training_set[feature], training_set[target])
    score = tmp.score(test_set[feature],test_set[target])
    print("최적화 n_estimator 값 : {}".format(N))
    print("예측률 : {}".format(score))
    
    # 만약 기존의 모델보다 더 예측률이 높다면, 최신화합니다
    if score>best_score:
        best_score = score
        best_N = N
        best_rf_model = tmp
        
print("최적화 n_estimator 값 : {}".format(best_N))
print("예측률 : {}".format(best_score))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



최적화 n_estimator 값 : 4000
예측률 : 0.9042194822640276


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



최적화 n_estimator 값 : 4500
예측률 : 0.9035964279289123


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



최적화 n_estimator 값 : 5000
예측률 : 0.9043838468558979


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



최적화 n_estimator 값 : 5500
예측률 : 0.9042545445216248


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



In [74]:
#best_score = 0
#for N in range(13,21):
#    tmp = RandomForestRegressor(n_estimators=4500, max_depth = N)
#    tmp.fit(training_set[feature], training_set[target])
#    score = tmp.score(test_set[feature],test_set[target])
#    
#    # 만약 기존의 모델보다 더 예측률이 높다면, 최신화합니다
#    if score>best_score:
#        best_score = score
#        best_N = N
#        best_rf_model = tmp
        
#print("최적화 max_depth 값 : {}".format(best_N))
#print("예측률 : {}".format(best_score))

In [75]:
#제출

In [146]:
forest.fit(training[feature], training[target])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=5000, n_jobs=2, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [147]:
prediction =forest.predict(test[feature])

In [148]:
prediction = np.array(list(map(lambda x: math.exp(x)-1,prediction)))

In [149]:
submission = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction})

In [150]:
submission.to_csv("Submission39.csv")